### 2 Analyzing GMO
NOTE: the analysis for GMWAX and GMGEX is done side-by-side throughout parts 1, 2, and 3, so code is not repeated in part 4 (there is only a discussion of the key differences between the two strategies there).

This section utilizes data in the file gmo_data.xlsx. Convert total returns to excess returns using the risk‑free rate.

1. Performance (GMWAX & GMGEX). Compute mean, volatility, and Sharpe ratio for GMWAX over three samples:
- inception → 2011
- 2012 → present
- inception → present

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

df = pd.read_excel('gmo_analysis_data.xlsx', sheet_name='total returns').set_index('date')
rf = pd.read_excel('gmo_analysis_data.xlsx', sheet_name='risk-free rate').set_index('date')
xr = df.sub(rf['TBill 3M'], axis=0)

In [6]:
def performance(sample):
    mu  = sample.mean() * 12
    vol = sample.std() * np.sqrt(12)
    return pd.DataFrame({'Mean': mu, 'Vol': vol, 'Sharpe': mu / vol})

res = {k: performance(v) for k, v in {'1996-2011': xr.loc[:'2011-12-31'], '2012-2025': xr.loc['2012-01-01':], '1996-2025': xr}.items()}
pd.concat(res, names=['Sample', 'Asset']).round(4)

Mean     Vol  Sharpe
Sample    Asset                        
1996-2011 SPY   -0.2759  0.1765 -1.5629
          GMWAX -0.2653  0.1319 -2.0118
          GMGEX -0.3155  0.1645 -1.9184
2012-2025 SPY   -0.0375  0.1527 -0.2455
          GMWAX -0.1233  0.1096 -1.1252
          GMGEX -0.1593  0.2317 -0.6876
1996-2025 SPY   -0.1618  0.1689 -0.9584
          GMWAX -0.1974  0.1233 -1.6012
          GMGEX -0.2408  0.2004 -1.2013

Has the mean, vol, and Sharpe changed much since the case?
1. GMWAX:   
The mean, volatility, and Sharpe ratio of GMWAX have all improved significantly, with particularly better performance in the 2012-2025 period.

2. GMGEX:   
For GMGEX, while its mean and Sharpe ratio have improved, its volatility has risen significantly, resulting in substantial changes in its risk characteristics.




2. Tail risk (GMWAX & GMGEX). For all three samples, analyze extreme scenarios:
- minimum return
- 5th percentile (VaR‑5th)
- maximum drawdown (compute on total returns, not excess returns)

In [7]:
def tailrisk(sample):
    min = sample.min()
    var = sample.quantile(0.05)
    dd = sample.apply(lambda x: (1+x).cumprod().div((1+x).cumprod().cummax()).sub(1))
    return pd.DataFrame({'Min Return': min, 'VaR (5%)': var, 'Max Drawdown': dd.min()})
# Using df here for total returns (as opposed to excess)
res2 = {k: tailrisk(v) for k, v in {'1996-2011': df.loc[:'2011-12-31'], '2012-2025': df.loc['2012-01-01':], '1996-2025': df}.items()}
pd.concat(res2, names=['Sample', 'Asset']).round(4)

Min Return  VaR (5%)  Max Drawdown
Sample    Asset                                    
1996-2011 SPY       -0.1652   -0.0795       -0.5080
          GMWAX     -0.1451   -0.0440       -0.2936
          GMGEX     -0.1512   -0.0797       -0.5556
2012-2025 SPY       -0.1246   -0.0607       -0.2393
          GMWAX     -0.1150   -0.0369       -0.2168
          GMGEX     -0.6587   -0.0653       -0.7374
1996-2025 SPY       -0.1652   -0.0744       -0.5080
          GMWAX     -0.1451   -0.0404       -0.2936
          GMGEX     -0.6587   -0.0752       -0.7618

(a) Does GMWAX/GMGEX have high or low tail‑risk as seen by these stats?
1. GMWAX:   
Based on the tail-risk statistics, GMWAX has low tail-risk.
2. GMGEX:   
Based on the tail-risk statistics, GMGEX has high tail-risk.

(b) Does that vary much across the two subsamples?
1. GMWAX:   
The tail-risk profile varies noticeably across subsamples.
2. GMGEX:   
The tail-risk profile varies

3. Market exposure (GMWAX & GMGEX). For all three samples, regress excess returns of GMWAX/GMGEX on excess returns of SPY:
- report estimated alpha, beta, and R²

In [8]:
def reg(sample):
    rows = []
    for asset in ['GMWAX', 'GMGEX']:
        y = sample[asset]
        X = sm.add_constant(sample['SPY'])
        model = sm.OLS(y, X, missing='drop').fit()
        rows.append([asset, model.params['const'], model.params['SPY'], model.rsquared])
        
    return pd.DataFrame(rows, columns=['Asset', 'alpha', 'beta', 'R2']).set_index('Asset')

res3 = {k: reg(v) for k, v in {'1996-2011': xr.loc[:'2011-12-31'], '2012-2025': xr.loc['2012-01-01':], '1996-2025': xr}.items()}
pd.concat(res3, names=['Sample', 'Asset']).round(4)

alpha    beta      R2
Sample    Asset                        
1996-2011 GMWAX -0.0079  0.6195  0.6878
          GMGEX -0.0076  0.8147  0.7645
2012-2025 GMWAX -0.0083  0.6294  0.7693
          GMGEX -0.0108  0.8042  0.2811
1996-2025 GMWAX -0.0081  0.6224  0.7271
          GMGEX -0.0092  0.8040  0.4588

- is GMWAX/GMGEX a low‑beta strategy? has that changed since the case?
1. GMWAX:   
GMWAX’s beta is 0.6195 (1996-2011), 0.6294 (2012-2025), and 0.6224 (1996-2025), all well below 1. Thus, it is a low-beta strategy, and its beta has not changed significantly across subsamples.


2. GMGEX:   
GMGEX’s beta is 0.8147 (1996-2011), 0.8042 (2012-2025), and 0.804 (1996-2025), all below 1. So, it is a low-beta strategy, and its beta has not changed significantly across subsamples either.


- does GMWAX/GMGEX provide alpha? has that changed across subsamples?
1. GMWAX:   
GMWAX’s alpha is -0.0079 (1996-2011), -0.0083 (2012-2025), and -0.0081 (1996-2025), all negative. Thus, it does not provide positive alpha, and its alpha has not changed significantly across subsamples.


2. GMGEX:   
GMGEX’s alpha is -0.0076 (1996-2011), -0.0108 (2012-2025), and -0.0092 (1996-2025), all negative. So, it does not provide positive alpha. However, its alpha became more negative in 2012-2025, so it has changed across subsamples to some extent.

4. Compare GMWAX and GMGEX. What are key differences between the two strategies?

- Risk & Return: GMWAX has lower tail risk and better Sharpe ratio; GMGEX has higher tail risk and worse Sharpe ratio.

- Market Exposure: GMWAX has lower beta and higher R² ; GMGEX has higher beta and lower R² in later periods .

- Excess Return: Neither has positive alpha; GMGEX has worse alpha stability.